In [118]:
from transformers import (
    AutoTokenizer, AutoModelForMaskedLM, 
)

# import nlpaug.augmenter.char as nac
# import nlpaug.augmenter.word as naw
# import nlpaug.augmenter.sentence as nas
# import nlpaug.flow as nafc
# from nlpaug.util import Action

# from googletrans import Translator
# import translators as ts

from collections import Counter
import re, math, random, json
from copy import deepcopy
from tqdm import tqdm
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split
import pandas as pd

from module.preprocess import decorate_form, decorate_acd_pair, decorate_asc_pair, decorate_acd_pair_split, decorate_asc_pair_split
from module.utils import count_tags, make_token_classification_pair, remove_props, get_filter

import demoji

# from cleantext import clean
# from pykospacing import Spacing
# from hanspell import spell_checker

In [119]:
def jsonload(fname, encoding="utf-8"):
    with open(fname, encoding=encoding) as f:
        j = json.load(f)

    return j


# json 개체를 파일이름으로 깔끔하게 저장
def jsondump(j, fname):
    with open(fname, "w", encoding="UTF8") as f:
        json.dump(j, f, ensure_ascii=False)

# jsonl 파일 읽어서 list에 저장
def jsonlload(fname, encoding="utf-8"):
    json_list = []
    with open(fname, encoding=encoding) as f:
        for line in f.readlines():
            json_list.append(json.loads(line))
    return json_list

# Load Raw Data

In [120]:
train_json = './dataset/nikluge-sa-2022-train.jsonl'
dev_json = './dataset/nikluge-sa-2022-dev.jsonl'
test_json = './dataset/nikluge-sa-2022-test.jsonl'

train = jsonlload(train_json)
dev = jsonlload(dev_json)
test = jsonlload(test_json)
train = pd.DataFrame(train)
dev = pd.DataFrame(dev)
test = pd.DataFrame(test)

# train = pd.read_json(train_json, lines=True)
# dev = pd.read_json(dev_json, lines=True)
# test = pd.read_json(test_json, lines=True)

train = train.drop(2319)
dev = dev.drop(1692)

# Declare Stuff to use

In [121]:
### new
entity_property_pair = [
    '본품#가격', '본품#다양성', '본품#디자인', '본품#인지도', '본품#일반', '본품#편의성', '본품#품질',
    '브랜드#가격', '브랜드#디자인', '브랜드#인지도', '브랜드#일반', '브랜드#품질',
    '제품 전체#가격', '제품 전체#다양성', '제품 전체#디자인', '제품 전체#인지도', '제품 전체#일반', '제품 전체#편의성', '제품 전체#품질',
    '패키지/구성품#가격', '패키지/구성품#다양성', '패키지/구성품#디자인', '패키지/구성품#일반', '패키지/구성품#편의성', '패키지/구성품#품질'

]
more_tokens = ['&name&', '&affiliation&', '&social-security-num&', '&tel-num&', '&card-num&', '&bank-account&', '&num&', '&online-account&']

tf_id_to_name = ['True', 'False']
tf_name_to_id = {tf_id_to_name[i]: i for i in range(len(tf_id_to_name))}

polarity_id_to_name = ['positive', 'negative', 'neutral']
polarity_name_to_id = {polarity_id_to_name[i]: i for i in range(len(polarity_id_to_name))}

# Count

In [122]:
# total = pd.concat([train, dev])
# total['stratified'] = total.annotation.apply(lambda x: x[0][0])
# # count_tags(total, entity_property_pair)

In [123]:
# indexes = []
# for idx, row in total.iterrows():
#     for annotation in row.annotation:
#         if annotation[0] in ['본품#인지도', '패키지/구성품#가격']:
#             indexes.append(row.id)
# total['checker'] = total.id.apply(lambda x: True if x in indexes else False)
# appendables = total[total.checker == True]

In [124]:
# appendables.to_json('temp.json')

In [125]:
# filter = [x for x in entity_property_pair if x not in ['본품#인지도', '패키지/구성품#가격']]
# total = remove_props(total, filter)
# # count_tags(total, entity_property_pair)

In [126]:
# train, dev, _, _ = train_test_split(total, total['stratified'], test_size=0.2, random_state=42,  stratify=total['stratified'])
# train.reset_index(inplace=True, drop=True)
# dev.reset_index(inplace=True, drop=True)
# appendables = pd.read_json('temp.json')
# train = pd.concat([train, appendables]).reset_index(drop=True)

In [127]:
# count_tags(train, entity_property_pair)

In [128]:
# count_tags(dev, entity_property_pair)

# Filter entity_property_pair and Drop rows accordingly

In [129]:
# FILTER_MODE = False

In [130]:
# if FILTER_MODE == True:
#     filter = get_filter()
#     train = remove_props(train, filter)
#     dev = remove_props(dev, filter)
# len(train), len(dev)

# Preprocess

## Cleansing

### Before

In [131]:
# for el in train.sample(n=5).sentence_form:
#     print(el)

In [132]:
# train.sentence_form = train.sentence_form.apply(preprocess)
# dev.sentence_form = dev.sentence_form.apply(preprocess)
# test.sentence_form = test.sentence_form.apply(preprocess)
# total = pd.concat([train, dev])

### Test

In [133]:
# case = total.sentence_form.str.contains('r[^A-Za-z0-9가-힣\s]+', case=False, flags=0, na=None, regex=True)
# for e in total[case].sentence_form:
#     print(e)

### After

In [134]:
# for i, row in total[['id', 'sentence_form']].sample(n=5).iterrows():
#     print(row.id, '\t', row.sentence_form)

In [135]:
# total['check'] = total.sentence_form.str.find('OO')
# for row in total[total.check > -1].sentence_form:
#     print(row)
#     break

In [136]:
# total

## Reformat

In [137]:
len(entity_property_pair)

25

In [138]:
decorate_form, decorate_acd_pair, decorate_asc_pair, decorate_acd_pair_split, decorate_asc_pair_split

(<function module.preprocess.decorate_form(form)>,
 <function module.preprocess.decorate_acd_pair(entity)>,
 <function module.preprocess.decorate_asc_pair(entity, sentiment)>,
 <function module.preprocess.decorate_acd_pair_split(entity)>,
 <function module.preprocess.decorate_asc_pair_split(entity, sentiment)>)

In [139]:
def reformat(df):
    ep =[]
    p = []
    for index, row in df.iterrows():
        utterance = row.sentence_form
        id = row.id
        
        form = utterance
        # form = decorate_form(utterance)

        for pair in entity_property_pair:
            isPairInOpinion = False
            if pd.isna(utterance):
                break
            for annotation in row.annotation:
                entity_property = annotation[0]
                sentiment = annotation[2]
                if entity_property == pair:
                    acd_pair = entity_property
                    ep_append = [id, form, acd_pair, tf_name_to_id['True']]
                    ep.append(ep_append)
                    p.append([id, utterance, acd_pair, polarity_name_to_id[sentiment]])
                    isPairInOpinion = True
                    break
            if isPairInOpinion is False:
                acd_pair = pair
                ep_append = [id, form, acd_pair, tf_name_to_id['False']]
                ep.append(ep_append)
    return ep, p

In [140]:
len(train), len(dev)

(3000, 2793)

In [141]:
ep_train, p_train = reformat(train)
ep_dev, p_dev = reformat(dev)

ep_train = pd.DataFrame(ep_train, columns=['id', 'form', 'pair', 'labels'])
ep_dev = pd.DataFrame(ep_dev, columns=['id', 'form', 'pair', 'labels'])

p_train = pd.DataFrame(p_train, columns=['id', 'form', 'pair', 'labels'])
p_dev = pd.DataFrame(p_dev, columns=['id', 'form', 'pair', 'labels'])

len(ep_train), len(ep_dev), len(p_train), len(p_dev)

(75000, 69825, 3196, 3002)

### Counting

In [142]:
print('binary_multi: ', end=''), print(len(ep_train), len(ep_dev), len(p_train), len(p_dev))
ep_train = ep_train.drop_duplicates()
ep_dev = ep_dev.drop_duplicates()
p_train = p_train.drop_duplicates()
p_dev = p_dev.drop_duplicates()
print('\nafter drop_duplicates\n')
print('binary_multi: ', end=''), print(len(ep_train), len(ep_dev), len(p_train), len(p_dev))

binary_multi: 75000 69825 3196 3002

after drop_duplicates

binary_multi: 75000 69825 3196 3002


(None, None)

In [143]:
p_train

,id,form,pair,labels
0,nikluge-sa-2022-train-00001,둘쨋날은 미친듯이 밟아봤더니 기어가 헛돌면서 틱틱 소리가 나서 경악.,본품#품질,1
1,nikluge-sa-2022-train-00002,"이거 뭐 삐꾸를 준 거 아냐 불안하고, 거금 투자한 게 왜 이래.. 싶어서 정이 확...",본품#품질,1
2,nikluge-sa-2022-train-00003,간사하게도 그 이후에는 라이딩이 아주 즐거워져서 만족스럽게 탔다.,제품 전체#일반,0
3,nikluge-sa-2022-train-00004,샥이 없는 모델이라 일반 도로에서 타면 노면의 진동 때문에 손목이 덜덜덜 떨리고 이...,제품 전체#일반,2
4,nikluge-sa-2022-train-00005,안장도 딱딱해서 엉덩이가 아팠는데 무시하고 타고 있다.,본품#일반,1
...,...,...,...,...
3191,nikluge-sa-2022-train-02997,(슬픔),제품 전체#가격,1
3192,nikluge-sa-2022-train-02998,보드랍고 괜찮다!,제품 전체#품질,0
3193,nikluge-sa-2022-train-02999,#일본 유니클로 질이 우리나라보다 좋은 것 같으면 기분 탓인가.......,브랜드#일반,2
3194,nikluge-sa-2022-train-03000,마지막으로 귀여워서 집어온 모자.,제품 전체#디자인,0


### Validate Here

In [144]:
# dfs = [ep_train, ep_dev, p_train, p_dev]
# for df in dfs:
#     for idx, row in df.iterrows():
#         print(row.id, '\n',
#             row.form, '\n',
#             row.pair, '\n',
#             row.labels,  '\n',)
#         if idx == 999:
#             break

### Save

In [145]:
DATA_V = 'uncleaned_v13'
save_path = f'./dataset/{DATA_V}'
print(save_path)

./dataset/uncleaned_v13


In [117]:
!mkdir -p {save_path}

train.to_csv(f'{save_path}/raw_train.csv', index=False, encoding='utf-8-sig')
dev.to_csv(f'{save_path}/raw_dev.csv', index=False, encoding='utf-8-sig')
test.to_csv(f'{save_path}/raw_test.csv', index=False, encoding='utf-8-sig')

ep_train.to_csv(f'{save_path}/ce_train.csv', index=False, encoding='utf-8-sig')
ep_dev.to_csv(f'{save_path}/ce_dev.csv', index=False, encoding='utf-8-sig')
p_train.to_csv(f'{save_path}/pc_train.csv', index=False, encoding='utf-8-sig')
p_dev.to_csv(f'{save_path}/pc_dev.csv', index=False, encoding='utf-8-sig')